### Retriever

In [1]:
from langchain_community.vectorstores import FAISS as VectorStore
from langchain_community.embeddings import GPT4AllEmbeddings

embeddings = GPT4AllEmbeddings()
store = VectorStore.load_local("../../retrieve/vector_store", embeddings, allow_dangerous_deserialization=True)
retriever = store.as_retriever(search_kwargs={"k": 4})

bert_load_from_file: gguf version     = 2
bert_load_from_file: gguf alignment   = 32
bert_load_from_file: gguf data offset = 695552
bert_load_from_file: model name           = BERT
bert_load_from_file: model architecture   = bert
bert_load_from_file: model file type      = 1
bert_load_from_file: bert tokenizer vocab = 30522


### Model

In [2]:
import dotenv
dotenv.load_dotenv()

True

In [3]:
from langchain_community.llms.huggingface_pipeline import HuggingFacePipeline

model_id= "tiiuae/falcon-7b-instruct"

llm = HuggingFacePipeline.from_model_id(
    model_id=model_id,
    task="text-generation",
    pipeline_kwargs={"max_new_tokens": 10},
)

'''
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id, device_map="auto")
pipe = pipeline("text-generation", model=model, tokenizer=tokenizer, max_new_tokens=10)
llm = HuggingFacePipeline(pipeline=pipe)
'''

2024-04-08 18:26:06.257630: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


tokenizer_config.json:   0%|          | 0.00/287 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.73M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/281 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.05k [00:00<?, ?B/s]

...

pytorch_model-00002-of-00002.bin:   0%|          | 0.00/4.48G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


generation_config.json:   0%|          | 0.00/117 [00:00<?, ?B/s]

'\nfrom transformers import AutoModelForCausalLM, AutoTokenizer, pipeline\n\ntokenizer = AutoTokenizer.from_pretrained(model_id)\nmodel = AutoModelForCausalLM.from_pretrained(model_id, device_map="auto")\npipe = pipeline("text-generation", model=model, tokenizer=tokenizer, max_new_tokens=10)\nllm = HuggingFacePipeline(pipeline=pipe)\n'

### Chains
https://python.langchain.com/docs/modules/memory/adding_memory_chain_multiple_inputs/

In [4]:
from langchain.chains.question_answering import load_qa_chain
from langchain.memory import ConversationBufferMemory
from langchain_core.prompts import PromptTemplate

template = """You are a chatbot having a conversation with a human.

Given the following extracted parts of a long document and a question, create a final answer.

{context}

{chat_history}
Human: {human_input}
Chatbot:"""

prompt = PromptTemplate(
    input_variables=["chat_history", "human_input", "context"], template=template
)
memory = ConversationBufferMemory(memory_key="chat_history", input_key="human_input")

chain = load_qa_chain(llm=llm, chain_type="stuff", memory=memory, prompt=prompt)

In [5]:
query = "How install the chatbots"
docs = retriever.invoke(query)

chain.invoke({"input_documents": docs, "human_input": query}) #, return_only_outputs=True) #117m 0.2s

{'input_documents': [Document(page_content="The chatbots are in their respective folders with an application launcher `autorun.sh` to install each of them without specific knowledge.  \nOn Mac, open the terminal and type:\n```shell\ncd\n```\nDrag the **`folder`** containing the file `autorun.sh`, then press the Enter key (↩︎).  \n_If you have done it correctly, the **`~`** between your machine's name (`name@MacBook-Pro-of-Name`) and the **`%`** sign should display the name of the `folder` instead._  \nExecute the following line of code by pressing the Enter key (↩︎):\n```shell\nsh autorun.sh\n```\nWait a moment, the model should open in your default web browser.", metadata={'Header 1': 'Models', 'Header 2': 'Installation', 'Header 3': 'Streamlit & FastAPI'}),
  Document(page_content='The subfolders listed below contain key steps in our research for creating a functional, convenient, and maintainable chatbot.', metadata={'Header 1': 'Onboarding Bot Model', 'Header 2': 'Structure'}),
  D